In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction
Improvement in mental health and stress management is key to enhance job performance productivity and more importantly, to lighten the burden of mental and physical health conditons boosting well-being amongst adults.
This analysis is aimed to look into strategies workplaces has taken to address mental issues and its impact.

# 1. set up environment and load data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
survey = pd.read_csv('/kaggle/input/mental-health-in-tech-2016/mental-heath-in-tech-2016_20161114.csv')

# 2. Investigate, data cleaning and processing
* Data has 1433 rows and 63 columns ( 1433 observations and 63 questions/attributes)
* rename columns

let's take a look at how our data is looking

In [ ]:
survey.shape
survey.head()

now lets rename columns

In [ ]:
# look at column names
cols = list(survey.columns)
cols
# rename columns
columns_renamed = ['self_employ_cat', 'employee_count', 'tech_comp_cat', 'tech_role_cat', 'mh_coverage_cat',
                 'mh_coverage_aware_cat', 'mh_employer_discus_cat', 'employer_mhresource_cat','employee_anonymity_cat', 'medical_leave_cat', 
                  'mh_employer_discus_negimpact_cat','ph_employer_discus_negimpact_cat', 'mh_cowork_discus_cat', 'mh_directsup_discus_cat','mh_vs_ph_employer',
                  'mh_coworker_discus_conseq','mh_coverage_cat2', 'aware_online_mhresoure_cat', 'employer_diag&reveamh_toclient_cat','employer_revealmh_toclient_neg_conseq_cat',
                  'employer_diag&reveamh_tocowork_cat','employer_revealmh_tocowork_neg_conseq_cat','mh_prod_impact_cat','mh_prod_impact_percent','prev_employer_cat',
                  'prevemployer_mh_benefit_cat','mh_coverage_aware_prev_cat','mh_employer_discus_prev_cat','employer_mhresource_prev_cat','employee_anonymity_prev_cat',
                  'mh_employer_discus_negimpact_prev_cat','ph_employer_discus_negimpact_prev_cat','mh_cowork_discus_prev_cat','mh_directsup_discus_prev_cat','mh_vs_ph_employer_prev',
                  'mh_coworker_discus_conseq_prev','future_ph_specification','ph_why/whynot','future_mh_specification','mh_why/whynot',
                  'mh_hurt_carrer','mh_neg_view_cowork','mh_sharing_friends/fam_cat','mh_bad_response_workplace','mh_for_others_bad_response_workplace',
                  'mh_family_hist','mh_disorder_past','mh_disorder_current','mh_diag_conditons','mh_maybe_conditions',
                  'mh_diag_professional','mh_prof_diag_conditons','mh_sought_prof_treatment','mh_eff_on_work_when_treat_fair','mh_eff_on_work_when_not_treat_fair',
                  'age','gender','country_residency','US_state_territory_live','country_work',
                  'US_state_territory_work','work_position','work_remote']
survey.columns = columns_renamed

In [ ]:
survey.head()

look for wrong/impossible column value

In [ ]:
# look at number of unique values
# show without folding
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(survey.nunique())

created unique_val_df to keep track of unique column values 

In [ ]:
# unique_val = {}
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     for col in survey:
#         print(col)
#         print(survey[col].unique())
#         print(type(survey[col].unique()))
#         print(survey[col].unique().size)
#         unique_val[col] = survey[col].unique()
#         print(unique_val[col])
# unique_val_df = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in unique_val.items()]))
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     for column in unique_val_df: # this loops the dataframe by column
#         print(column)
#         print (unique_val_df[column].unique())
#         print(survey[column].unique().size)
#         print('\n')

Gender
* there are 71 unique values

In [ ]:
unique_val_df['gender'].unique().size

* recode gender, employee count
* fix age outlier by replace any age not in range 18 and 75

In [ ]:
# recode female == 1, male == 2, other == 3
survey['gender'].replace(to_replace = ['Female','female','I identify as female.','Female assigned at birth ',
                                      'F','Woman','fm','f','Cis female ','woman','female/woman','Cisgender Female',
                                       'fem', 'Female (props for making this a freeform field, though)','Cis-woman',
                                       'female-bodied; no feelings about gender','AFAB','Female ', ' Female','female '
                                      ], value = 'Female', inplace = True)
survey['gender'].replace(to_replace = ['Male','male','M','m','man','Cis male','Male.', 'Male (cis)','Man',
                                      'Sex is male','cis male','Malr','Dude',"I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
                                       'mail','M|','male ','Cis Male', 'cisdude', 'cis man','MALE',' Male', 'Male '                                      ], value = 'Male', inplace = True)
survey['gender'].replace(to_replace = ['Bigender','non-binary','Transitioned, M2F','Genderfluid (born female)',
                                      'Other/Transfeminine','Female or Multi-Gender Femme','Androgynous',
                                       'male 9:1 female, roughly', 'Other','nb masculine', 'none of your business',
                                       'genderqueer','Human', 'Genderfluid', 'Enby','genderqueer woman','mtf',
                                       'Queer','Agender', 'Fluid','Male/genderqueer','Nonbinary','human', 'Unicorn', 
                                        'Male (trans, FtM)','Genderqueer','Genderflux demi-girl', 'Transgender woman'
                                      ], value = 'Other', inplace = True)
# recode employee_count
survey['employee_count'].replace(to_replace = ['More than 1000'], value = '>1000', inplace = True)

# recode age
age_mean = survey[(survey['age'] >= 15) | (survey['age'] <= 75)]['age'].mean()
survey['age'].replace(to_replace = survey[(survey['age'] < 15) | (survey['age'] > 75)]['age'].tolist(),
value = age_mean, inplace = True)


In [ ]:
# check if all unique values are considered
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(survey['gender'])
survey['gender'].unique()

look for N/A column values

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (16,4))
sns.heatmap(data = survey.isna())

take a look at the number of missing values from each column

In [ ]:
# check if more than half of observations have na values
# all cols have more than half observations w/o na values
# half_obs = 1443//2
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(survey.isna().sum())

drop the following columns with more than half observations missing

       'tech_role_cat', 'mh_coverage_cat2', 'aware_online_mhresoure_cat',
       'employer_diag&reveamh_toclient_cat',
       'employer_revealmh_toclient_neg_conseq_cat',
       'employer_diag&reveamh_tocowork_cat',
       'employer_revealmh_tocowork_neg_conseq_cat', 'mh_prod_impact_cat',
       'mh_prod_impact_percent', 'mh_for_others_bad_response_workplace',
       'mh_diag_conditons', 'mh_maybe_conditions', 'mh_prof_diag_conditons'
       
now we are left with 50 attributes

In [ ]:
# survey has 1433 observations, drop columns with more than half of observations missing
more_than_half_na = (survey.isna().sum() / survey.shape[0] >= 0.5).tolist()

drop_cols = survey.columns[more_than_half_na]
print(drop_cols)
survey.drop(labels = drop_cols, axis = 1, inplace = True)
survey.shape

here is the number of missing values in remaining columns

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(survey.isna().sum())

 Only age has numerical values

In [ ]:
# employee_counts is categorical
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(survey['employee_count'])

there are 464 observations w/o any missing data

In [ ]:
# drop a row if any of its column has missing values
drop_all_na = survey.dropna(how = 'any')

check to see if drop_all_na has no missing value at all

In [ ]:

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(drop_all_na.isna().sum())

In [ ]:
plt.figure(figsize = (16,4))
sns.heatmap(data = drop_all_na.isna())

# Is mental health issues a topic people can talk about more freely?


* Discussing with their employer:
> Employees tend to think that they can talk about mental health issues more freely with their current employer
than their previous ones without negative consequence

Note: took away answers that were not specific: 'I don't know', 'Maybe', 

In [ ]:
fig, ax = plt.subplots(figsize = (18,18), ncols = 2, nrows = 2)
# fig.suptitle('Do you think that discussing a mental health disorder with your current vs previous employer would have negative consequences?',fontsize=20)
fig.suptitle('discussing mental health disorder with employer',fontsize=20)

neutral = ''
colors = ['#92C6FF', '#97F0AA', '#FF9F9A', '#D0BBFF', '#FFFEA3', '#B0E0E6']

neutral = 'I don\'t know'
labels = list(drop_all_na['mh_employer_discus_negimpact_prev_cat'].value_counts().index.drop(labels = neutral))
sizes = drop_all_na['mh_employer_discus_negimpact_prev_cat'].value_counts()
explode = [0, 0.05, 0]
sizes = remove_idk(neutral, sizes)
ax[0][0].pie(sizes, labels=labels, autopct='%1.1f%%',colors = colors, explode = explode)
ax[0][0].set_title('neg consequences\n(with previous employers)',pad = 16, fontsize = 16)
ax[0][0].legend(loc='upper right')

neutral = 'Maybe'
labels_2 = list(drop_all_na['mh_employer_discus_negimpact_cat'].value_counts().index.drop(labels = neutral))
sizes_2 = remove_idk(neutral,drop_all_na['mh_employer_discus_negimpact_cat'].value_counts())
colors = ['#FF9F9A', '#97F0AA']
explode = [0, 0.05]
ax[0][1].pie(sizes_2, labels=labels_2, autopct='%1.1f%%',colors = colors, explode = explode)
ax[0][1].set_title('neg consequences\n(with current employers)',pad = 16, fontsize = 16)
ax[0][1].legend(loc='upper right')

neutral = 'I don\'t know'
labels_3 = list(drop_all_na['mh_directsup_discus_prev_cat'].value_counts().index.drop(labels = neutral))
sizes_3 = remove_idk(neutral, drop_all_na['mh_directsup_discus_prev_cat'].value_counts())
colors = ['#92C6FF', '#FF9F9A','#97F0AA']
explode = [0, 0, 0.05]
ax[1][0].pie(sizes_3, labels=labels_3, autopct='%1.1f%%',colors = colors, explode = explode)
ax[1][0].set_title('willingness to discuss mh issues \n(with previous direct supervisor)',pad = 16, fontsize = 16)
ax[1][0].legend(loc='upper right')

neutral = 'Maybe'
labels_4 = list(drop_all_na['mh_directsup_discus_cat'].value_counts().index.drop(labels = neutral))
sizes_4 = remove_idk(neutral, drop_all_na['mh_directsup_discus_cat'].value_counts())
colors = ['#97F0AA', '#FF9F9A']
explode = [0.05, 0]
ax[1][1].pie(sizes_4, labels=labels_4, autopct='%1.1f%%',colors = colors, explode = explode)
ax[1][1].set_title('willingness to discuss mh issues \n(with current direct supervisor)',pad = 16, fontsize = 16)
ax[1][1].legend(loc='upper right')

# from matplotlib change legend title / text
# plt.setp(ax[1][1].get_legend().get_title(), fontsize='32') # for legend title
plt.setp(ax[0][0].get_legend().get_texts(), fontsize='11') # for legend text
plt.setp(ax[0][1].get_legend().get_texts(), fontsize='11')
plt.setp(ax[1][0].get_legend().get_texts(), fontsize='11')
plt.setp(ax[1][1].get_legend().get_texts(), fontsize='11')

plt.tight_layout()
plt.show()

Would you feel comfortable discussing a mental health disorder with your coworkers?

* Discussing with their coworker:
> * More people believe that ther will not be a negative consequence discussing a mental disorder with their employer
> * Yet there are a good amount of people who are on the fence

take a look at related survey questions:
1. Would you feel comfortable discussing a mental health disorder with your coworkers?
> variable name: mh_cowork_discus_cat
2. Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?
> variable name: mh_coworker_discus_conseq

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize = (15,10), ncols = 2, nrows =1)
fig.suptitle ('discussing mental health disorder with co-workers (current)', fontsize=20)
neutral = ''
colors = ['#92C6FF', '#97F0AA', '#FF9F9A', '#D0BBFF', '#FFFEA3', '#B0E0E6']

neutral = 'Maybe'
labels = list(drop_all_na['mh_cowork_discus_cat'].value_counts().index.drop(labels = neutral))
sizes = drop_all_na['mh_cowork_discus_cat'].value_counts()
sizes = remove_idk(neutral, sizes)
explode = [0, 0.05]
colors = ['#FF9F9A', '#97F0AA',]
ax[0].pie(sizes, labels = labels, autopct = '%1.1f%%', colors = colors, explode = explode)
ax[0].set_title ('Are you willing?')
ax[0].legend(loc = 'upper right')

labels_2 = list(drop_all_na['mh_coworker_discus_conseq'].value_counts().index)
sizes_2 = drop_all_na['mh_coworker_discus_conseq'].value_counts()
explode = [0, 0.05]
ax[1].pie(sizes_2, labels = labels_2, autopct = '%1.1f%%', colors = colors, explode = explode)
# * Have you heard of or observed negative consequences for co-workers that are open about mh issues?
ax[1].set_title ('neg consequences \nfor co-workers that are open about mh issues?')
ax[1].legend(loc = 'upper right')

plt.tight_layout()
plt.show()

Current workplace
* Most people think that there are no observed negative consequences for co-workers who openly talk about mental health issues.
* Yet over half of the employees are not comfortable with talking about mental health issues in workplace.
* Employees might want to keep dicussions professional in workplace and choose to not talk about something private like mental health issues.

More questions
3. Would you have been willing to discuss a mental health issue with your previous co-workers?
> variable name: mh_cowork_discus_prev_cat
4. Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?
> variable name: mh_coworker_discus_conseq_prev
5. Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?
> variable name: mh_neg_view_cowork

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize = (15,10), ncols = 2, nrows =1)
fig.suptitle ('discussing mental health disorder with co-workers (previous)', fontsize=20)
neutral = ''
colors = ['#92C6FF', '#97F0AA', '#FF9F9A', '#D0BBFF', '#FFFEA3', '#B0E0E6']


# labels = list(drop_all_na['mh_coworker_discus_conseq_prev'].value_counts().index)
# sizes = drop_all_na['mh_coworker_discus_conseq_prev'].value_counts()

# explode = [0, 0, 0.05]
# colors = ['#FF9F9A', '#92C6FF', '#97F0AA']
# ax[1].pie(sizes, labels=labels, autopct='%1.1f%%',colors = colors, explode = explode)
# ax[1].set_title('neg consequences',pad = 16, fontsize = 16)
# ax[1].legend(loc='upper right')


labels_2 = list(drop_all_na['mh_cowork_discus_prev_cat'].value_counts().index)
sizes_2 = drop_all_na['mh_cowork_discus_prev_cat'].value_counts()
drop_all_na['mh_cowork_discus_prev_cat']
# colors = ['#92C6FF', '#FF9F9A','#97F0AA']
# explode = [0, 0, 0.05]
# ax[0].pie(sizes_2, labels=labels_2, autopct='%1.1f%%',colors = colors, explode = explode)
# ax[0].set_title('Are you willing?',pad = 16, fontsize = 16)
# ax[0].legend(loc='upper right')
# plt.tight_layout()
# plt.show()

Previous workplace
* 

* Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?
> variable name: mh_neg_view_cowork

In [ ]:
drop_all_na['mh_neg_view_cowork'].value_counts()

In [ ]:
# mh_neg_view_cowork

colors = ['#92C6FF', '#97F0AA', '#FF9F9A', '#D0BBFF', '#FFFEA3', '#B0E0E6']
o = ['Yes, they do','Yes, I think they would','Maybe','No, I don\'t think they would','No, they do not']
g2 = sns.countplot(x="mh_neg_view_cowork", data=drop_all_na, palette = colors, order = o, ti)
g2.set_xticklabels(g2.get_xticklabels(), rotation = 30, ha = "right")
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title='Gender')
plt.tight_layout()
plt.show()

In [ ]:
x = drop_all_na['mh_coworker_discus_conseq'].value_counts()
x

In [ ]:
y = drop_all_na['mh_cowork_discus_cat'].value_counts()
y

In [ ]:
# drop_all_na['mh_coworker_discus_conseq'].index
g1 = sns.barplot(x = drop_all_na['mh_coworker_discus_conseq'].index,
                 y = drop_all_na['mh_cowork_discus_cat'],
             data = drop_all_na, capsize=.2)
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title='Gender')

In [ ]:
g2 = sns.countplot(x='mh_employer_discus_negimpact_cat',
                  hue ='employee_count', data = drop_all_na, order=['Yes', 'No', 'Maybe'],
                   hue_order = ['1-5', '6-25', '26-100', '100-500', '500-1000','>1000'])
g2.set_xlabel('Do you think that discussing a mental health disorder with your employer would have negative consequences?')